In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install awscli

     |████████████████████████████████| 3.7 MB 15.9 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 8.2 MB 32.7 MB/s 
     |████████████████████████████████| 547 kB 62.2 MB/s 
     |████████████████████████████████| 138 kB 55.8 MB/s 
  Created wheel for botocore: filename=botocore-1.21.48-py3-none-any.whl size=7938740 sha256=802f1743db4028bc1da23a14b5115ad9bbc56490a68a71569236e1c3a1b2aa63
  Stored in directory: /root/.cache/pip/wheels/43/47/8d/118e8b8a0c53e8d6c024d07ce366f07331cd7c01bc1872a7b1
Successfully built botocore
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [1]:
text = '''
[default]
aws_access_key_id = ##################
aws_secret_access_key = ##################
region = us-east-2
'''
path = "/content/drive/MyDrive/config/awscli.ini"
with open(path, 'w') as f:
   f.write(text)
!cat /content/drive/MyDrive/config/awscli.ini

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/config/awscli.ini'

In [4]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/My Drive/config/awscli.ini


In [6]:
!aws s3 ls

2021-09-23 17:07:53 myspacenetdataset


In [7]:
!aws s3 cp s3://spacenet-dataset/spacenet/SN3_roads/tarballs/SN3_roads_train_AOI_2_Vegas.tar.gz . 

download: s3://spacenet-dataset/spacenet/SN3_roads/tarballs/SN3_roads_train_AOI_2_Vegas.tar.gz to ./SN3_roads_train_AOI_2_Vegas.tar.gz


In [8]:
!pip install osmnx
!pip install apls

     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████████████████████| 6.3 MB 54.0 MB/s 
     |████████████████████████████████| 10.3 MB 33.2 MB/s 
     |████████████████████████████████| 994 kB 46.9 MB/s 
     |████████████████████████████████| 994 kB 47.5 MB/s 
     |████████████████████████████████| 62 kB 834 kB/s 
     |████████████████████████████████| 15.4 MB 50 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompat

     |████████████████████████████████| 77 kB 4.3 MB/s 
     |████████████████████████████████| 564 kB 38.3 MB/s 
  ERROR: Failed building wheel for GDAL
  Running setup.py clean for GDAL
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6109 sha256=db8e50e49fc135efb80259105902b85e46db662f5862ff8dc280f7f286c490ee
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm
Failed to build GDAL
  Attempting uninstall: GDAL
    Found existing installation: GDAL 2.2.2
    Uninstalling GDAL-2.2.2:
      Successfully uninstalled GDAL-2.2.2
    Running setup.py install for GDAL ... error
  Rolling back uninstall of GDAL
  Moving to /usr/local/lib/python3.7/dist-packages/GDAL-2.2.2.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~DAL-2.2.2.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/__pycache__/gdal.cpython-37.pyc
   from /tmp/pip-uninstall-rg1_97nq/gdal.cpython-37.pyc
  Moving to /

In [9]:
import tarfile

In [10]:
file = tarfile.open("/content/SN3_roads_train_AOI_2_Vegas.tar.gz")
file.extractall("/content/")  
file.close()

In [11]:
import os
import sys
import time
import argparse
import pandas as pd
import numpy as np
import subprocess
from osgeo import gdal, ogr, osr
import geopandas as gpd
#import rasterio as rio
import shutil
from collections import defaultdict
import osmnx as ox
import cv2

In [12]:
rescale = {
    '2': {
        1: [25.48938322, 1468.79676441],
        2: [145.74823054, 1804.91911021],
        3: [155.47927199, 1239.49848332]
    },
    '4': {
        1: [79.29799666, 978.35058431],
        2: [196.66026711, 1143.74207012],
        3: [170.72954925, 822.32387312]
    },
    '3': {
        1: [46.26129032, 1088.43225806],
        2: [127.54516129, 1002.20322581],
        3: [141.64516129, 681.90967742]
    },
    '5': {
        1: [101.63250883, 1178.05300353],
        2: [165.38869258, 1190.5229682 ],
        3: [126.5335689, 776.70671378]
    }
}

In [13]:
def convert_to_8Bit(inputRaster, outputRaster,
                           outputPixType='Byte',
                           outputFormat='GTiff',
                           rescale_type='rescale',
                           percentiles=[2, 98]):
    '''
    Convert 16bit image to 8bit
    rescale_type = [clip, rescale]
        if clip, scaling is done strictly between 0 65535 
        if rescale, each band is rescaled to a min and max 
        set by percentiles
    '''

    srcRaster = gdal.Open(inputRaster)
    cmd = ['gdal_translate', '-ot', outputPixType, '-of', 
           outputFormat]
    
    # iterate through bands
    for bandId in range(srcRaster.RasterCount):
        bandId = bandId+1
        band = srcRaster.GetRasterBand(bandId)
        if rescale_type == 'rescale':
            bmin = band.GetMinimum()        
            bmax = band.GetMaximum()
            # if not exist minimum and maximum values
            if bmin is None or bmax is None:
                (bmin, bmax) = band.ComputeRasterMinMax(1)
            # else, rescale
            band_arr_tmp = band.ReadAsArray()
            bmin = np.percentile(band_arr_tmp.flatten(), 
                                 percentiles[0])
            bmax= np.percentile(band_arr_tmp.flatten(), 
                                percentiles[1])
        elif isinstance(rescale_type, dict):
            bmin, bmax = rescale_type[bandId]
        else:
            bmin, bmax = 0, 65535

        cmd.append('-scale_{}'.format(bandId))
        cmd.append('{}'.format(bmin))
        cmd.append('{}'.format(bmax))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))

    cmd.append(inputRaster)
    cmd.append(outputRaster)
    print("Conversin command:", cmd)
    subprocess.call(cmd)
    
    return


In [14]:
def create_buffer_geopandas(geoJsonFileName, bufferDistanceMeters=2, 
                          bufferRoundness=1, projectToUTM=True):
    '''
    Create a buffer around the lines of the geojson. 
    Return a geodataframe.
    '''
    if os.stat(geoJsonFileName).st_size == 0:
        return []
    inGDF = gpd.read_file(geoJsonFileName)

    # set a few columns that we will need later
    inGDF['type'] = inGDF['road_type'].values            
    inGDF['class'] = 'highway'  
    inGDF['highway'] = 'highway'  
    
    if len(inGDF) == 0:
        return []

    # Transform gdf Roadlines into UTM so that Buffer makes sense
    if projectToUTM:
        tmpGDF = ox.project_gdf(inGDF)
    else:
        tmpGDF = inGDF

    gdf_utm_buffer = tmpGDF

    # perform Buffer to produce polygons from Line Segments
    gdf_utm_buffer['geometry'] = tmpGDF.buffer(bufferDistanceMeters,
                                                bufferRoundness)

    gdf_utm_dissolve = gdf_utm_buffer.dissolve(by='class')
    gdf_utm_dissolve.crs = gdf_utm_buffer.crs

    if projectToUTM:
        gdf_buffer = gdf_utm_dissolve.to_crs(inGDF.crs)
    else:
        gdf_buffer = gdf_utm_dissolve

    return gdf_buffer


In [15]:
def gdf_to_array(gdf, im_file, output_raster, burnValue=150):
    
    '''
    Turn geodataframe to array, save as image file with non-null pixels 
    set to burnValue
    '''

    NoData_value = 0      # -9999

    gdata = gdal.Open(im_file)
    
    # set target info
    target_ds = gdal.GetDriverByName('GTiff').Create(output_raster, 
                                                     gdata.RasterXSize, 
                                                     gdata.RasterYSize, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform(gdata.GetGeoTransform())
    
    # set raster info
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(gdata.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())
    
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)
    
    outdriver=ogr.GetDriverByName('MEMORY')
    outDataSource=outdriver.CreateDataSource('memData')
    tmp=outdriver.Open('memData',1)
    outLayer = outDataSource.CreateLayer("states_extent", raster_srs, 
                                         geom_type=ogr.wkbMultiPolygon)
    # burn
    burnField = "burn"
    idField = ogr.FieldDefn(burnField, ogr.OFTInteger)
    outLayer.CreateField(idField)
    featureDefn = outLayer.GetLayerDefn()
    for geomShape in gdf['geometry'].values:
        
        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(ogr.CreateGeometryFromWkt(geomShape.wkt))
        outFeature.SetField(burnField, burnValue)
        outLayer.CreateFeature(outFeature)
        outFeature = 0
    
    gdal.RasterizeLayer(target_ds, [1], outLayer, burn_values=[burnValue])
    outLayer = 0
    outDatSource = 0
    tmp = 0
        
    return 

In [16]:
def get_road_buffer(geoJson, im_vis_file, output_raster, 
                              buffer_meters=2, burnValue=1, 
                              bufferRoundness=6, 
                              plot_file='', figsize=(6,6), fontsize=6,
                              dpi=800, show_plot=False, 
                              verbose=False):    
    '''
    Get buffer around roads defined by geojson and image files.
    Calls create_buffer_geopandas() and gdf_to_array().
    Assumes in_vis_file is an 8-bit RGB file.
    Returns geodataframe and ouptut mask.
    '''
    
     
    gdf_buffer = create_buffer_geopandas(geoJson,
                                         bufferDistanceMeters=buffer_meters,
                                         bufferRoundness=bufferRoundness, 
                                         projectToUTM=True)    

        
    # create label image
    if len(gdf_buffer) == 0:
        mask_gray = np.zeros(cv2.imread(im_vis_file,0).shape)
        cv2.imwrite(output_raster, mask_gray)        
    else:
        gdf_to_array(gdf_buffer, im_vis_file, output_raster, 
                                          burnValue=burnValue)
    # load mask
    mask_gray = cv2.imread(output_raster, 0)
    """
    # make plots
    if plot_file:
        # plot all in a line
        if (figsize[0] != figsize[1]):
            fig, (ax0, ax1, ax2, ax3) = plt.subplots(1,4, figsize=figsize)#(13,4))
        # else, plot a 2 x 2 grid
        else:
            fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2,2, figsize=figsize)
    
        # road lines
        try:
            gdfRoadLines = gpd.read_file(geoJson)
            gdfRoadLines.plot(ax=ax0, marker='o', color='red')
        except:
            ax0.imshow(mask_gray)
        ax0.axis('off')
        ax0.set_aspect('equal')
        ax0.set_title('Roads from GeoJson', fontsize=fontsize)
                
        # first show raw image
        im_vis = cv2.imread(im_vis_file, 1)
        img_mpl = cv2.cvtColor(im_vis, cv2.COLOR_BGR2RGB)
        ax1.imshow(img_mpl)
        ax1.axis('off')
        ax1.set_title('8-bit RGB Image', fontsize=fontsize)
        
        # plot mask
        ax2.imshow(mask_gray)
        ax2.axis('off')
        ax2.set_title('Roads Mask (' + str(np.round(buffer_meters)) \
                                   + ' meter buffer)', fontsize=fontsize)
     
        # plot combined
        ax3.imshow(img_mpl)    
        # overlay mask
        # set zeros to nan
        z = mask_gray.astype(float)
        z[z==0] = np.nan
        # change palette to orange
        palette = plt.cm.gray
        #palette.set_over('yellow', 0.9)
        palette.set_over('lime', 0.9)
        ax3.imshow(z, cmap=palette, alpha=0.66, 
                norm=matplotlib.colors.Normalize(vmin=0.5, vmax=0.9, clip=False))
        ax3.set_title('8-bit RGB Image + Buffered Roads', fontsize=fontsize) 
        ax3.axis('off')
        
        #plt.axes().set_aspect('equal', 'datalim')

        plt.tight_layout()
        plt.savefig(plot_file, dpi=dpi)
        if not show_plot:
            plt.close()
        """
    return mask_gray, gdf_buffer

In [18]:
buffer_meters = 2
burnValue = 255
path_apls = r'/content/drive/MyDrive/Ishana/dataset'
path_outputs = os.path.join(path_apls, 'vegas_dataset', 'masks{}m'.format(buffer_meters))
path_images_8bit = os.path.join(path_apls, 'vegas_dataset', 'images') 
for d in [path_outputs, path_images_8bit]:
        shutil.rmtree(d, ignore_errors=True)
        os.makedirs(d, exist_ok=True)

In [19]:
path_data = '/content/AOI_2_Vegas'
test_data_name = os.path.split(path_data)[-1]
test_data_name = '_'.join(test_data_name.split('_')[:3]) + '_'
path_images_raw = os.path.join(path_data, 'PS-RGB')
path_labels = os.path.join(path_data, 'geojson_roads')

In [20]:
#iterate through raw images, convert to 8bit & craete masks
im_files = os.listdir(path_images_raw)
m = defaultdict(list)
for im_file in im_files:
  if not im_file.endswith('.tif'):
    continue
  name_root = im_file.split('_')[-1].split('.')[0]
  
  #create 8bit image
  im_file_raw = os.path.join(path_images_raw, im_file)
  im_file_out = os.path.join(path_images_8bit, name_root +'.tif')
  rescale_type = test_data_name.split('_')[1]

  if not os.path.isfile(im_file_out):
    convert_to_8Bit(im_file_raw, im_file_out, 
                    outputPixType = 'Byte', outputFormat = 'GTiff',
                    rescale_type = rescale[rescale_type], percentiles = [2,98])
  
  #determine output files
  label_file = os.path.join(path_labels,'SN3_roads_train_' + test_data_name + 'geojson_roads_' + name_root +'.geojson')
  label_file_tot = os.path.join(path_labels, label_file)
  output_raster = os.path.join(path_outputs, name_root + '.png')

  if not os.path.isfile(label_file):
    continue
  #create masks
  if not os.path.isfile(output_raster):
    print("\nname_root:", name_root)
    print("  output_raster:", output_raster)
    mask, gdf_buffer = get_road_buffer(label_file_tot, im_file_out, output_raster, 
                                       buffer_meters = buffer_meters, burnValue = burnValue,
                                       plot_file = None, figsize = (6,6),
                                       fontsize=8, dpi=200,
                                       show_plot=False, verbose=False)
    
  for k,v in m.items():
    print(test_data_name, k, np.mean(v, axis=0))


Conversin command: ['gdal_translate', '-ot', 'Byte', '-of', 'GTiff', '-scale_1', '25.48938322', '1468.79676441', '0', '255', '-scale_2', '145.74823054', '1804.91911021', '0', '255', '-scale_3', '155.47927199', '1239.49848332', '0', '255', '/content/AOI_2_Vegas/PS-RGB/SN3_roads_train_AOI_2_Vegas_PS-RGB_img160.tif', '/content/drive/MyDrive/Ishana/dataset/vegas_dataset/images/img160.tif']

name_root: img160
  output_raster: /content/drive/MyDrive/Ishana/dataset/vegas_dataset/masks2m/img160.png
Conversin command: ['gdal_translate', '-ot', 'Byte', '-of', 'GTiff', '-scale_1', '25.48938322', '1468.79676441', '0', '255', '-scale_2', '145.74823054', '1804.91911021', '0', '255', '-scale_3', '155.47927199', '1239.49848332', '0', '255', '/content/AOI_2_Vegas/PS-RGB/SN3_roads_train_AOI_2_Vegas_PS-RGB_img376.tif', '/content/drive/MyDrive/Ishana/dataset/vegas_dataset/images/img376.tif']

name_root: img376
  output_raster: /content/drive/MyDrive/Ishana/dataset/vegas_dataset/masks2m/img376.png
Conversi